## Preprocessing

In [114]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [115]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [116]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [117]:
# Determine the number of unique values in each column.
# unique_value_counts = {x: len(application_df[x].unique()) for x in application_df.columns}
# print(unique_value_counts) # or

unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [118]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [119]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts[counts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [120]:
# Look at CLASSIFICATION value counts for binning
counts_binning = application_df['CLASSIFICATION'].value_counts()

In [121]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classification = counts_binning[counts_binning>1]
counts_classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [122]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list (counts_binning[counts_binning<100].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [123]:
# Convert categorical data to numeric with `pd.get_dummies`
# application_df = pd.get_dummies(application_df,dtype=float)
# Instead of specifying the dtype parameter as float, we can use dtype='int64' to save memory since the one-hot encoded variables are binary (i.e., 0 or 1) and can be represented using integers. 
# This will not change the output of the code, but it will make the code more efficient in terms of memory usage.
application_df = pd.get_dummies(application_df, dtype='int64')
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [124]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values

# drop 'IS SUCCESSFUL'
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [125]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)

(25724, 49)


## Compile, Train and Evaluate the Model

In [142]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features=len(X_train_scaled[0])
hidden_nodes_layer1=120
hidden_nodes_layer2=120
hidden_nodes_layer3=120
hidden_nodes_layer4=80
hidden_nodes_layer5=60
hidden_nodes_layer6=50
hidden_nodes_layer7=10
hidden_nodes_layer8=10
nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="relu"))
# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, input_dim=number_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer6, input_dim=number_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer7, input_dim=number_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer8, input_dim=number_input_features, activation="relu"))
# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_162 (Dense)           (None, 120)               6000      
                                                                 
 dense_163 (Dense)           (None, 120)               14520     
                                                                 
 dense_164 (Dense)           (None, 120)               14520     
                                                                 
 dense_165 (Dense)           (None, 80)                9680      
                                                                 
 dense_166 (Dense)           (None, 60)                4860      
                                                                 
 dense_167 (Dense)           (None, 50)                3050      
                                                                 
 dense_168 (Dense)           (None, 10)              

In [143]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [144]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)


Epoch 1/100
684/684 [==============================] - 4s 3ms/step - loss: 0.5712 - accuracy: 0.7179 - val_loss: 0.5504 - val_accuracy: 0.7349
Epoch 2/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5558 - accuracy: 0.7282 - val_loss: 0.5457 - val_accuracy: 0.7341
Epoch 3/100
684/684 [==============================] - 3s 4ms/step - loss: 0.5536 - accuracy: 0.7294 - val_loss: 0.5503 - val_accuracy: 0.7359
Epoch 4/100
684/684 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7307 - val_loss: 0.5420 - val_accuracy: 0.7359
Epoch 5/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7315 - val_loss: 0.5446 - val_accuracy: 0.7357
Epoch 6/100
684/684 [==============================] - 2s 4ms/step - loss: 0.5479 - accuracy: 0.7320 - val_loss: 0.5466 - val_accuracy: 0.7344
Epoch 7/100
684/684 [==============================] - 3s 5ms/step - loss: 0.5485 - accuracy: 0.7304 - val_loss: 0.5430 - val_accuracy: 0.7349

Epoch 58/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5346 - accuracy: 0.7381 - val_loss: 0.5420 - val_accuracy: 0.7362
Epoch 59/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7342 - val_loss: 0.5496 - val_accuracy: 0.7354
Epoch 60/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7371 - val_loss: 0.5458 - val_accuracy: 0.7388
Epoch 61/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7392 - val_loss: 0.5516 - val_accuracy: 0.7393
Epoch 62/100
684/684 [==============================] - 3s 4ms/step - loss: 0.5325 - accuracy: 0.7392 - val_loss: 0.5496 - val_accuracy: 0.7388
Epoch 63/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5324 - accuracy: 0.7394 - val_loss: 0.5491 - val_accuracy: 0.7388
Epoch 64/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5320 - accuracy: 0.7386 - val_loss: 0.5508 - val_accuracy:

In [145]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5794 - accuracy: 0.7292 - 588ms/epoch - 2ms/step
Loss: 0.5793757438659668, Accuracy: 0.7292128205299377


In [146]:
# Export our model to HDF5 file
# from google.colab import files

nn_model.save('AlphabetSoupCharity_Optimization.h5')
# files.download('AlphabetSoupCharity.h5')